# FINDATA 02 - 종목 부가정보

(stock_desc 테이블)

#### 이승준 http://fb.com/plusjune


In [41]:
import pandas as pd
from IPython.display import display

In [42]:
import sqlite3
conn = sqlite3.connect('findata.db')

In [44]:
# 테이블  읽기
df_master = pd.read_sql_query('SELECT * FROM stock_master', conn, index_col='code')
df_master.head()

,name,sector_code,sector
code,,,
060310,3S,032602,전자부품 제조업
095570,AJ네트웍스,126903,산업용 기계 및 장비 임대업
068400,AJ렌터카,126901,운송장비 임대업
006840,AK홀딩스,137105,"회사본부, 지주회사 및 경영컨설팅 서비스업"
054620,AP시스템,032902,특수 목적용 기계 제조업


# 태그 검색

* dt 태그를 검색 class='line-left'

In [45]:
import requests
from bs4 import BeautifulSoup

def get_func_templ(code):
    url = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + code
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")

    for s in soup.findAll('dt', {'class':'line-left'}):
        display( s )

display (get_func_templ('005930'))

<dt class=" line-left"><a href="../company/c1020001.aspx?cmp_cd=005930&amp;cn=">기업개요</a></dt>

<dt class=" line-left"><a href="../company/c1030001.aspx?cmp_cd=005930&amp;cn=">재무분석</a></dt>

<dt class=" line-left"><a href="../company/c1040001.aspx?cmp_cd=005930&amp;cn=">투자지표</a></dt>

<dt class=" line-left"><a href="../company/c1050001.aspx?cmp_cd=005930&amp;cn=">컨센서스</a></dt>

<dt class=" line-left"><a href="../company/c1060001.aspx?cmp_cd=005930&amp;cn=">업종분석</a></dt>

...

<dt class="line-left">PER <b class="num">11.74</b></dt>

<dt class="line-left">업종PER <b class="num">9.29</b></dt>

<dt class="line-left">PBR <b class="num">1.09</b></dt>

<dt class="line-left">배당수익률 <b class="num">1.63%</b></dt>

None

# 네이버 스크래핑 테스트

In [46]:
import requests
import time
from bs4 import BeautifulSoup

def get_sector(code):
    url = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + code
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"lxml")
    sector = ""

    # find KRX sector
    for s in soup.findAll('dt', {'class':'line-left'}):
        if s.text.find('KOSPI :') >= 0:
            sector = s.text.split(' : ')[1]
        elif s.text.find('KOSDAQ :') >= 0:
            sector = s.text.split(' : ')[1]

    return sector

display (get_sector('005930'))
display (get_sector('068270'))

'전기전자'

'제약'

# 포탈 종목 정보
* sector, wics, name_en
* desc, desc_date

In [47]:
import requests
import time
import json
from bs4 import BeautifulSoup

# sector, wics, name_en
def get_sector(code):
    name_en, sector, wics, market = 'nan', 'nan', 'nan', 'nan'
    url = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + code
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"lxml")
    td = soup.find('td', {'class':'cmp-table-cell td0101'})
    if td == None:
        return sector, wics, name_en
        
    dts = td.findAll('dt')

    # dts[1], name_en
    name_en = dts[1].text
    
    # dts[2], sector
    s = dts[2]
    if s.text.find('KOSPI :') >= 0:
        market = 'KOSPI'
        sector = s.text.split(' : ')[1]
    elif dts[2].text.find('KOSDAQ :') >= 0:
        market = 'KOSDAQ'
        sector = s.text.split(' : ')[1]
            
    # dts[3], wics
    s = dts[3]
    if s.text.find('WICS :') >= 0:
        wics = s.text.split(' : ')[1]

    return market, wics, name_en


# desc, desc_date
def get_desc(code):
    url = 'http://companyinfo.stock.naver.com/v1/company/cmpcomment.aspx'
 
    cmt_text = ' '
    cmt_date = time.strftime("%Y-%m-%d")
 
    if code[-1] == '5': # pre_order
        code = code[0:-1] + '0'
    if code[-1] == '7': # pre_order
        code = code[0:-1] + '0'
 
    headers = {'Host':'companyinfo.stock.naver.com'}
    r = requests.post(url, data={'cmp_cd': code}, headers=headers)
    if r.text == "":
        return (cmt_text, cmt_date)
 
    j = json.loads(r.text)
    cmt_date = j['dt'].replace('.', '-')
    cmts = j['data'][0]
    cmts = [cmts['COMMENT_1'], cmts['COMMENT_2'], cmts['COMMENT_3'],  cmts['COMMENT_4'], cmts['COMMENT_5'] ]
    cmt_text = '. '.join(cmts)
    return (cmt_text, cmt_date)


display( get_sector('005930') )
display( get_desc('005930'))

display (get_sector('204840'))
display (get_desc('204840'))


('KOSPI', '반도체와반도체장비', 'Samsung Electronics')

('1969년 설립되어, 본사를 거점으로 한국 및 CE, IM 부문 산하 해외 9개 지역총괄과 DS 부문 산하 해외 5개 지역총괄의 생산, 판매법인 등 159개의 동종업종을 영위하는 종속기업으로 구성된 글로벌 전자기업임. 주요사업은 제품의 특성에 따라 CE부문(TV,냉장고,세탁기 등)과 IM부문(스마트폰 등 HHP,네트워크시스템 등), DS부문(DRAM, NAND Flash, 모바일AP, LCD패널 등) 3개의 부문으로 나누어짐. IM부문은 프리미엄 브랜드 갤럭시(GALAXY)를 필두로 보급형 휴대폰까지 풀라인업을 유지하면서 S시리즈, Note 등의 Flagship스마트폰의 성공을 바탕으로 글로벌 스마트폰 1위 위상을 유지. 반도체 사업은 DRAM, NAND Flash 제품 등 초미세 공정 기술을 적용하여 경쟁사 대비 차별성 있는 제품의 생산과 원가 경쟁력을 지속적으로 확보하여 전세계 메모리 시장의 선두 자리를 지속적으로 유지. 매출구성은 IM 51.6%, 반도체 23.71%, CE 23.37%, DP 13.7% 등으로 구분',
 '2016-04-23')

('KOSDAQ', '미분류', 'IBKS No.2 Special Purpose Acquisition Company')

(' ', '2016-05-08')

In [48]:
# 테이블  읽기
df_desc = pd.read_sql_query('SELECT * FROM stock_desc', conn)
df_desc.head()

,code,name,telephone,address,market,wics,name_en,desc,desc_date
0,060310,3S,02-896-9474,서울특별시 금천구 시흥대로71길 30-1,,,,,
1,095570,AJ네트웍스,02-6240-0800,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)",,,,,
2,068400,AJ렌터카,1544-1600,서울특별시 구로구 서부샛길 822,,,,,
3,006840,AK홀딩스,02-6923-2921,서울특별시 구로구 구로중앙로 152(구로동),,,,,
4,054620,AP시스템,031-379-2700,경기도 화성시 동탄면 동탄산단8길 15-5,,,,,


In [ ]:
# df_desc
# name, telephone, address, market, code, wics, name_en, desc, desc_date, corp_type, market_cap, update_date

cur = conn.cursor()

for idx, row in df_desc.iterrows():
    if row['wics'] == '':
        print('sector', row['code'])
        market, wics, name_en = get_sector(row['code'])
        cur.execute('UPDATE stock_desc SET market=?, wics=?, name_en=? WHERE code=?', (market, wics, name_en, row['code']) )
        conn.commit()

    if row['desc_date'] == '':
        print('desc', row['code'])
        desc, desc_date = get_desc(row['code'])
        cur.execute('UPDATE stock_desc SET desc=?, desc_date=? WHERE code=?', (desc, desc_date, row['code']) )
        conn.commit()


In [50]:
cursor = conn.execute("SELECT * from stock_desc limit 5")
for row in cursor:
    print (row)

('060310', '3S', '02-896-9474', '서울특별시 금천구  시흥대로71길 30-1 ', 'KOSDAQ', '반도체와반도체장비', '3S Korea', '동사는 1991년 설립되어 2002년 코스닥시장에 상장. 반도체웨이퍼 캐리어박스와 냉동공조용 열량계, 환경시험설비 전문업체임. 웨이퍼 캐리어 사업은 국제규격인 SEMI Standard를 충족시키는 300mm 웨이퍼용 FOSB를 개발하여 2009년 10월부터 국내외 웨이퍼사와 반도체 칩메이커사에 공급 중. 450mm 웨이퍼캐리어 시장 선점을 위하여 2013년 6월 미국의 Entegris사와 한국 내 450mm 웨이퍼 캐리어 판매를 위한 파트너쉽 계약을 체결. 환경장치사업은 냉동공조용 설비 칼로리메타, 환경구현장치 등을 주력제품으로 생산하며 삼성전자, LG전자, 위니아만도, 한라공조 등에 납품하고 있음. 매출구성은 칼로리메타 외 56.89%, 웨이퍼캐리어 41.98%, 냉동부품 외 1.14% 등으로 구성', '2016-04-26')
('095570', 'AJ네트웍스', '02-6240-0800', '서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)', 'KOSPI', '도로와철도운송', 'AJ Networks', '2000년 2월 설립되었으며 2013년 12월 지배회사인 아주엘앤에프홀딩스(주)를 흡수 합병하였고, 자회사 등의 업무지원 사업을 영위하는 지주부문과 동사의 주요사업을 영위하는 사업부문으로 조직을 개편함. 주요 사업으로 파렛트, OA기기, 정보통신기기(계측기), 고소장비 등의 장단기 대여와 판매 및 유통사업을 영위하고 있음. 렌탈 산업에 특화된 AJ그룹의 지주사로 AJ렌터카(자동차), AJ토탈(냉장/냉동창고), AJ파크(주차장) 등의 자회사를 보유함. 현재는 OA기기와 더불어 모바일렌탈(통신서비스포함), 고소장비 정비기술과 파렛트 현장 영업력으로 시너지효과를 발휘 할 수 있는 포크리프트 렌탈사업에 진출하였음. 매출구성은 파렛트부문 39.36%, IT부문 46.05%, 고소장비부문 11

In [51]:
# 테이블  읽기
df = pd.read_sql_query('SELECT * FROM stock_desc', conn)
df.tail()

,code,name,telephone,address,market,wics,name_en,desc,desc_date
2045,010240,흥국,041-546-7771,충청남도 아산시 둔포면 아산밸리로 357,,,,,
2046,189980,흥국에프엔비,043-882-5666,충청북도 음성군 삼성면 하이텍산단로 116-23 &nbsp,,,,,
2047,000540,흥국화재,02-2002-6000,서울특별시 종로구 새문안로 68,,,,,
2048,003280,흥아해운,02-3449-3000,서울특별시 송파구 새말로5길 21 흥아해운 빌딩,,,,,
2049,037440,희림,02-3410-9219,서울특별시 강동구 상일로6길 39-,,,,,


# 전체 코드

In [ ]:
# stock_desc.py
# name, telephone, address, market, code, wics, name_en, desc, desc_date, corp_type, market_cap, update_date

import pandas as pd
import requests
import time
import json
from bs4 import BeautifulSoup
import sqlite3


# sector, wics, name_en
def get_sector(code):
    name_en, sector, wics, market = 'nan', 'nan', 'nan', 'nan'
    url = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + code
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"lxml")
    td = soup.find('td', {'class':'cmp-table-cell td0101'})
    if td == None:
        return sector, wics, name_en
        
    dts = td.findAll('dt')

    # dts[1], name_en
    name_en = dts[1].text
    
    # dts[2], sector
    s = dts[2]
    if s.text.find('KOSPI :') >= 0:
        market = 'KOSPI'
        sector = s.text.split(' : ')[1]
    elif dts[2].text.find('KOSDAQ :') >= 0:
        market = 'KOSDAQ'
        sector = s.text.split(' : ')[1]
            
    # dts[3], wics
    s = dts[3]
    if s.text.find('WICS :') >= 0:
        wics = s.text.split(' : ')[1]

    return market, wics, name_en


# desc, desc_date
def get_desc(code):
    url = 'http://companyinfo.stock.naver.com/v1/company/cmpcomment.aspx'
 
    cmt_text = ' '
    cmt_date = time.strftime("%Y-%m-%d")
 
    if code[-1] == '5': # pre_order
        code = code[0:-1] + '0'
    if code[-1] == '7': # pre_order
        code = code[0:-1] + '0'
 
    headers = {'Host':'companyinfo.stock.naver.com'}
    r = requests.post(url, data={'cmp_cd': code}, headers=headers)
    if r.text == "":
        return (cmt_text, cmt_date)
 
    j = json.loads(r.text)
    cmt_date = j['dt'].replace('.', '-')
    cmts = j['data'][0]
    cmts = [cmts['COMMENT_1'], cmts['COMMENT_2'], cmts['COMMENT_3'],  cmts['COMMENT_4'], cmts['COMMENT_5'] ]
    cmt_text = '. '.join(cmts)
    return (cmt_text, cmt_date)
    
    
if __name__ == "__main__":
    conn = sqlite3.connect('findata.db')
    cur = conn.cursor()

    df_desc = pd.read_sql_query('SELECT * FROM stock_desc', conn)

    for idx, row in df_desc.iterrows():
        if row['wics'] == '':
            print('sector', row['code'], row['name'])
            market, wics, name_en = get_sector(row['code'])
            print (market, wics, name_en)
            cur.execute('UPDATE stock_desc SET market=?, wics=?, name_en=? WHERE code=?', (market, wics, name_en, row['code']) )
            conn.commit()
    
        if row['desc_date'] == '':
            desc, desc_date = get_desc(row['code'])
            print (desc, desc_date)
            cur.execute('UPDATE stock_desc SET desc=?, desc_date=? WHERE code=?', (desc, desc_date, row['code']) )
            conn.commit()
        print()
        
    conn.close()
